In [1]:
import pandas as pd
import os
from cogfy_manager import CogfyClient
from urllib.parse import urljoin
import requests
from tqdm.notebook import tqdm
from dotenv import load_dotenv

In [2]:
load_dotenv()

df = pd.read_csv("../dataset/scripts_dataset.csv")
df

,pergunta,classe,resposta
0,Quais serviços estão disponíveis nos balcões d...,Geral – Atendimento Presencial,A plataforma gov.br está com uma iniciativa de...
1,Qual suporte vocês oferecem para problemas enc...,Geral - Problema em sistemas diversos,Escrevemos em atenção à sua solicitação para i...
2,Como faço para obter um arquivo com o registro...,Geral - Arquivo da conversa no chat,"Para solicitar o arquivo com esta conversa, pe..."
3,Qual é o número de telefone para atendimento s...,Geral - Atendimento telefônico,Não há canal para atendimento telefônico sobre...
4,Como posso deixar uma avaliação positiva sobre...,Geral - Elogios,Agradecemos o seu elogio! A equipe gov.br está...
...,...,...,...
74,Como faço para acessar os serviços de Imposto ...,Nível insuficiente - Imposto de Renda,Os serviços de Imposto de Renda (https://www.g...
75,Como um MEI pode acessar e melhorar o nível da...,Nível insuficiente - MEI,Os serviços de MEI são prestados pela Receita ...
76,Como posso acessar os serviços do Meu INSS se ...,Nível insuficiente - Meu INSS,Os serviços do Meu INSS (https://meu.inss.gov....
77,Como posso acessar e utilizar o sistema Regist...,Nível insuficiente - Registrato,O sistema Registrato (https://registrato.bcb.g...


In [3]:
k = os.environ["COGFY_API_KEY"]
api_url = "https://public-api.serpro.cogfy.com"
client = CogfyClient(base_url=api_url, api_key=k)

In [5]:
# Collection Gov.br
collection_id = client.get_collection_id("dataset_train")
collection_id

'a890f4f5-80b5-4c61-86d8-c447c7c4f6bd'

In [6]:
client.find_collection(collection_id)

Collection(id='a890f4f5-80b5-4c61-86d8-c447c7c4f6bd', name='dataset_train')

In [7]:
field_id = "dcc473a3-6e98-4fd3-87c4-67d0091674f9"
chat_id = "5c400d65-b89d-4d93-801e-dbee18a122bb"

In [9]:
collection_id

'a890f4f5-80b5-4c61-86d8-c447c7c4f6bd'

### Test run

In [19]:
params = {
    "1e4955ef-b251-4294-9169-e575acbe3ef3": {
        "type": "text",
        "text": {"value": "Orientações cadastro #32"}
    },
    "5d06f718-4e4c-47d1-991b-14a1f63bbd46": {
        "type": "text",
        "text": {"value": "como faço para mudar minha conta para ouro ?"}
    },
    "5748c4bb-4fac-48be-a6ba-5ddcbb196f40": {
        "type": "text",
        "text": {"value": "Para mudar sua Conta GOV.BR para o nível Ouro, você pode seguir estas orientações principais:"}
    }
}
client.create_record(collection_id=collection_id, properties=params)

{'id': 'fc4b3c62-3a66-47a7-bf8a-66bb763274fe'}

In [21]:
def generate_payload(categoria: str, pergunta: str, resposta_esperada: str):
    params = {
        "1e4955ef-b251-4294-9169-e575acbe3ef3": {
            "type": "text",
            "text": {"value": categoria}
        },
        "5d06f718-4e4c-47d1-991b-14a1f63bbd46": {
            "type": "text",
            "text": {"value": pergunta}
        },
        "5748c4bb-4fac-48be-a6ba-5ddcbb196f40": {
            "type": "text",
            "text": {"value": resposta_esperada}
        }
    }
    return params

In [22]:
for idx, row in tqdm(df.iterrows(), total=len(df)):
    properties = generate_payload(categoria=row.classe, pergunta=row.pergunta, resposta_esperada=row.resposta)
    client.create_record(collection_id=collection_id, properties=properties)


  0%|          | 0/79 [00:00<?, ?it/s]